***Лабораторная работа 2***

Постройте программу, которая в качестве входа принимает произвольную КС-грамматику G = (N, ∑, P, S) и преобразует ее в эквивалентную КС-грамматику G' = (N', ∑', P', S') без левой рекурсии ($$ ||) не содержащую недостижимых символов.

In [31]:
class Rule:
    def __init__(self, left: str, right:list[str]):
        self.left = left
        self.right = right.copy()

class Grammar:
    nonterminals = []
    terminals = []
    rules = []
    axiom = ''

    def __init__(self, nonterminals: list[str], terminals: list[str], rules: list[Rule], axiom: str):
        self.nonterminals = nonterminals
        self.terminals = terminals
        self.rules = rules
        self.axiom = axiom
    
    def print(self, std=True, md=True):
        output = ""
        output += "[{:}] [{:}] {:}\n".format(",".join(sorted(self.nonterminals)), ",".join(sorted(self.terminals)), self.axiom) 

        for nonterminals in sorted(self.nonterminals):
            rules = list(filter(lambda x: x.left == nonterminals, self.rules))
            if not len(rules):
                continue
            
            output += '\n' + ( nonterminals + " -> " + " | ".join(sorted(map(lambda x: " ".join(x.right) if len(x.right) else '\epsilon', rules))))
        if std:
            print(output)
        
        if md:
            output = "\n\n".join(map(lambda x: '$' + x.replace('->', '\\rightarrow') + '$' if len(x) else '', output.split('\n')))

        return output
    def show(self):
        import IPython

        return IPython.display.Markdown(self.print(std=False))
    @staticmethod
    def parse_grammar(text):
        nonterminals, t, a = text.split('\n')[0].split(' ')
        nonterminals = nonterminals[1:-1].split(',')
        t = t[1:-1].split(',')
        
        rules = []

        for string in text.split('\n')[1:]:
            if not len(string):
                continue
            
            left, right = string.split(' -> ')

            for rule in right.split(' | '):
                data = rule.split(' ')
                if data[0] == '\epsilon':
                    data = []
                rules.append(
                    Rule(left, data)
                )
        return Grammar(nonterminals, t, rules, a)

In [32]:
grammar = Grammar.parse_grammar(
"""[A,B,C,D] [a,b,c] A
A -> A a | a | B
B -> C b 
C -> B c
D -> A
""")
display(grammar.show())

$[A,B,C,D] [a,b,c] A$



$A \rightarrow A a | B | a$

$B \rightarrow C b $

$C \rightarrow B c$

$D \rightarrow A$

Устранение левой рекурсии (для всех)

In [33]:
def LR_elimination(g: Grammar):
    new_rules: list[Rule] = g.rules.copy()

    nts = g.nonterminals.copy()

    for i, i_nt in enumerate(g.nonterminals):
        for j_nt in g.nonterminals[:i]:
            rules = filter(lambda x: x.left == i_nt and len(x.right) and x.right[0] == j_nt, new_rules)
            for rule in rules:
                new_rules.remove(rule)
                m_rules = filter(lambda x: x.left == j_nt, new_rules)
                for m_rule in m_rules:
                    new_rules.append(
                        Rule(
                            i_nt, m_rule.right + rule.right[1:]
                        )
                    )
        rules = list(filter(lambda x: x.left == i_nt, new_rules))
        need_modify = False
        for rule in rules:
            if len(rule.right) and rule.right[0] == i_nt:
                need_modify = True
                break
                
        if need_modify:
            new_nt = i_nt + "'"
            nts += [new_nt]

            for rule in rules:
                new_rules.remove(rule)
                if len(rule.right) and rule.right[0] == i_nt:
                    new_rules += [
                        Rule(new_nt, rule.right[1:]),
                        Rule(new_nt, rule.right[1:] + [new_nt]),
                    ]
                else:
                    new_rules += [
                        Rule(rule.left, rule.right.copy()),
                        Rule(rule.left, rule.right + [new_nt]),
                    ]
        
    return Grammar(nts, g.terminals.copy(), new_rules, g.axiom)

Устранение недостижимых символов (по варианту 1)

In [34]:
def UnRS_elimination(g: Grammar):
    queue = [g.axiom]
    nts = [ g.axiom]

    new_rules = []

    while len(queue):
        nonterminals = queue.pop(0)

        for rule in filter(lambda x: x.left == nonterminals, g.rules):
            new_rules.append(rule)
            for symbol in rule.right:
                if symbol in g.nonterminals and symbol not in nts:
                    nts.append(symbol)
                    queue.append(symbol)

    return Grammar(nts, g.terminals, new_rules, g.axiom)

In [35]:
print('----------------------   Исходная грамматика   ---------------------------')
display(grammar.show())
print('------------------   Грамматика без левой рекурсии   ---------------------')
display(LR_elimination(grammar).show())
print('--------------   Грамматика без недостижимых символов   ------------------')
display(UnRS_elimination(grammar).show())

----------------------   Исходная грамматика   ---------------------------


$[A,B,C,D] [a,b,c] A$



$A \rightarrow A a | B | a$

$B \rightarrow C b $

$C \rightarrow B c$

$D \rightarrow A$

------------------   Грамматика без левой рекурсии   ---------------------


$[A,A',B,C,C',D] [a,b,c] A$



$A \rightarrow B | B A' | a | a A'$

$A' \rightarrow a | a A'$

$B \rightarrow C b $

$C' \rightarrow b  c | b  c C'$

$D \rightarrow B A' | a | a A'$

--------------   Грамматика без недостижимых символов   ------------------


$[A,B,C] [a,b,c] A$



$A \rightarrow A a | B | a$

$B \rightarrow C b $

$C \rightarrow B c$